In [ ]:
import graph_generator as gg
import obm_dp as dp
import numpy as np
from util import diff

%load_ext autoreload
%autoreload 2

In [ ]:
m = 4; n = 4
config = {
    'graph_type': 'GEOM',
    'threshold': 0.5,
    'scaling': 1/ np.sqrt(2)
}
A = gg.sample_bipartite_graph(m, n, **config)
p = [0.75 for _ in range(m)]

cache = dp.cache_stochastic_opt(A, p)

In [ ]:
graphs = []

def update(dict, u):
    del dict[u]
    for (key, val) in dict.items():
        if val > u:
            dict[key] -= 1
    

def generate_examples(A, cache, p):
    graphs = []

    p = list(np.copy(p))
    m, n = A.shape
    offline_nodes = frozenset(np.arange(n))
    reduced_nodes = frozenset(np.arange(n))
    reduced_of_node = dict(zip(range(n), range(n)))
    reduced_A = np.copy(A)

    for t in range(m):
        hint, neighbor_mask = dp.one_step_stochastic_opt(A, offline_nodes, t, cache)
        graphs.append(gg._to_pyg(reduced_A, p, hint, neighbor_mask))
        choice = cache[t][offline_nodes][1]
        
        offline_nodes = diff(offline_nodes, choice)
        
        if choice != -1:
            reduced_nodes = diff(reduced_nodes, reduced_of_node[choice])
            update(reduced_of_node, choice)
       
        reduced_A = reduced_A[1:, :]
        reduced_A = np.take(reduced_A, list(reduced_nodes), axis=1)
        reduced_nodes = frozenset(np.arange(len(reduced_nodes)))
        p.pop()

    return graphs
           

generate_examples(A, cache, p)

